# Régression logistique

## 1 Régularisation de de Tikhonov

### Question 1.1

$$
f_1: (w_0, w) \mapsto \frac{1}{n}\sum_{i=1}^nlog(1 + e^{-y_i(x_i^Tw + w_0)}) + \frac{\rho}{2}\|w\|_2^2
$$

#### Gradient de $f_1$

$$
\frac{\partial f_1}{\partial w_0}(w_0, w) = \frac{1}{n}\sum_{i=1}^n\frac{-y_ie^{-y_i(x_i^Tw+w_0)}}{1 + e^{-y_i(x_i^T+w_0)}}
$$

$$
\frac{\partial f_1}{\partial w}(w_0, w) = \frac{1}{n}\sum_{i=1}^n\frac{-y_ie^{-y_i(x_i^Tw+w_0)}}{1 + e^{-y_i(x_i^T+w_0)}}x_i + \rho w
$$

$$
\nabla f_1(w_0, w) = \begin{pmatrix}\frac{1}{n}\sum_{i=1}^n\frac{-y_ie^{-y_i(x_i^Tw+w_0)}}{1 + e^{-y_i(x_i^T+w_0)}} \\  \frac{1}{n}\sum_{i=1}^n\frac{-y_ie^{-y_i(x_i^Tw+w_0)}}{1 + e^{-y_i(x_i^T+w_0)}}x_i + \rho w\end{pmatrix}
$$

#### Matrice hessienne de $f_1$

$$
\frac{\partial^2f_1}{\partial w_0^2}(w_0, w) = \frac{1}{n}\sum_{i=1}^ny_i^2\frac{e^{-y_i(x_i^Tw+w_0)}}{(1 + e^{-y_i(x_i^T+w_0)})^2}
$$

$$
\frac{\partial^2f_1}{\partial w^2}(w_0, w) = \frac{1}{n}\sum_{i=1}^ny_i^2\frac{e^{-y_i(x_i^Tw+w_0)}}{(1 + e^{-y_i(x_i^T+w_0)})^2}x_ix_i^T + \rho I_p
$$

$$
\frac{\partial^2{f_1}}{\partial w_0\partial w}(w_0, w) = \frac{1}{n}\sum_{i=1}^ny_i^2\frac{e^{-y_i(x_i^Tw+w_0)}}{(1 + e^{-y_i(x_i^T+w_0)})^2}x_i^T
$$ $$
\frac{\partial^2{f_1}}{\partial w\partial w_0}(w_0, w) = \frac{1}{n}\sum_{i=1}^ny_i^2\frac{e^{-y_i(x_i^Tw+w_0)}}{(1 + e^{-y_i(x_i^T+w_0)})^2}x_i
$$

$$
\text{Soit H la matrice hessienne de $f_1$ : }H(f_1) = \begin{bmatrix}\frac{\partial^2f_1}{\partial w_0^2} & \frac{\partial^2{f_1}}{\partial w_0\partial w} \\ \frac{\partial^2{f_1}}{\partial w\partial w_0} & \frac{\partial^2f_1}{\partial w^2}\end{bmatrix} = \begin{bmatrix}\frac{1}{n}\sum_{i=1}^ny_i^2\frac{e^{-y_i(x_i^Tw+w_0)}}{(1 + e^{-y_i(x_i^T+w_0)})^2} & \frac{1}{n}\sum_{i=1}^ny_i^2\frac{e^{-y_i(x_i^Tw+w_0)}}{(1 + e^{-y_i(x_i^T+w_0)})^2}x_i^T \\ \frac{1}{n}\sum_{i=1}^ny_i^2\frac{e^{-y_i(x_i^Tw+w_0)}}{(1 + e^{-y_i(x_i^T+w_0)})^2}x_i & \frac{1}{n}\sum_{i=1}^ny_i^2\frac{e^{-y_i(x_i^Tw+w_0)}}{(1 + e^{-y_i(x_i^T+w_0)})^2}x_ix_i^T + \rho  I_p\end{bmatrix}
$$

#### Convexité de $f_1$

$\|.\|_2^2$ est convexe.  
Soit $f: x \mapsto log(1 + e^x)$ ; $\frac{\partial^2{f}}{\partial x^2}(x) = \frac{e^x}{(1 + e^x)^2} > 0$ donc $f$ est convexe. De plus, $(w_0, w) \mapsto x_i^Tw + w_0$ est une fonction affine donc convexe.

Ainsi, $f_1$ est convexe.